## Sentiment Analysis

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 32.2 MB/s 
     |████████████████████████████████| 596 kB 52.7 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 6.6 MB 72.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import transformers
from transformers import pipeline

In [ ]:
# Sentiment model 1 불러오기
classifier = \
    pipeline('sentiment-analysis', model='snunlp/KR-FinBert-SC')

Downloading:   0%|          | 0.00/881 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/387M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/372 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/140k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/287k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Sentiment model 2 불러오기
classifier2 = \
    pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def sentiment():
  for i in range(4):
    # 파일 불러오기
    eng_file_list = ["oilprice", "war", "inf", "ex"]
    df = pd.read_csv(eng_file_list[i]+"_news.csv")

    # 기존 기사 데이터프레임에 sentiment model 1 값 추가하기
    # Label: positive, neutral, negative
    # Score: Label일 확률(0~1)
    df['Label1'] = ""
    df["Score1"] = pd.Series([], dtype=float)
    df["Label1_Total"] = pd.Series([], dtype=float)

    text = df['Title'].tolist()
    class_ = classifier(text)
    for j in range(len(df)):
      df['Label1'][j] = class_[j]['label']
      df['Score1'][j] = class_[j]['score']

    # Sentiment model 1은 postive, neutral, negative로 나와있는데, 이걸 1, 0.5, 0으로 변경
    df["Label1_Num"] = pd.Series([], dtype=float)

    conditionlist = [
        (df['Label1'] == 'positive') ,
        (df['Label1'] == 'neutral'), 
        (df['Label1'] == 'negative')]
    choicelist = [1, 0.5, 0]
    df['Label1_Num'] = np.select(conditionlist, choicelist, default='Not Specified')

    # 위에서 숫자로 변경한 값을 Score과 합하여 최종 sentiment score 구하기
    for j in range(len(df)):
      df['Label1_Total'][j] = float(df['Label1_Num'][j]) * float(df['Score1'][j])


    # 기존 기사 데이터프레임에 sentiment model 2 값 추가
    # Label: 1 star ~ 5 stars
    # Score: Label일 확률(0~1)

    df['Label2'] = ""
    df["Score2"] = pd.Series([], dtype=float)
    df["Label2_Total"] = pd.Series([], dtype=float)

    text = df['Title'].tolist()
    class_ = classifier2(text)

    for j in range(len(df)):
      df['Label2'][j] = int(class_[j]['label'][0])
      df['Score2'][j] = class_[j]['score']
    df["Label2_Total"] = df['Label2'].astype(float)

    # 데이터프레임 슬라이싱해서 저장
    df2 = df[["Date", "Topic", "Title", "Label1", "Label1_Num", "Score1", "Label1_Total" ,"Label2", "Score2", "Label2_Total"]]
    df3 = df[["Date", "Topic", "Title", "Label1_Total" , "Label2_Total"]]

    # groupby를 위하여 datetime으로 한 번 더 변경
    df3['Date']=pd.to_datetime(df3['Date'])

    # 날짜별로 groupby 한 뒤 평균 구해서 csv파일로 저장
    group = df3.sort_values(['Date'],ascending=True).groupby('Date')
    df4 = group.mean()
    df4.to_csv(eng_file_list[i] + '_sen_mean.csv')

In [ ]:
sentiment()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to b